In [ ]:
import numpy as np
import cv2 as cv
from IPython.display import display
from PIL import Image

from random import randint
from glob import glob


class Card:

    def __init__(self, suit, value, image):
        self.suit = suit
        self.value = value
        self.name = f"{value}{suit}"
        self.image = image

        xMin, xMax = 8, 40
        yMin, yMax = 16, 91
        self.upperLeftPoly = self._buildRectanglePoly(
            xMin, yMin, xMax, yMax)

        height, width, _ = self.image.shape
        self.lowerRightPoly = self._buildRectanglePoly(
            width-xMax, height-yMax,
            width-xMin, height-yMin
        )

    def _buildRectanglePoly(self, xMin: int, yMin: int, xMax: int, yMax: int):
        return [(xMin, yMin), (xMin, yMax), (xMax, yMax), (xMax, yMin)]

    def _displayPoly(self, img, points: list, color: list, thickness: int):
        endPoints = list(points[1:] + points[:1])
        for p1, p2 in zip(points, endPoints):
            cv.line(img, p1, p2, color, thickness)

    def _transformPoly(self, points: list, matrix):
        result = list(matrix @ (*a, 1) for a in points)
        result = list(list(int(b) for b in a[:2]) for a in result)
        return result

    def display(self):
        # image = cv.cvtColor(self.image, cv.COLOR_BGR2RGB)
        image = self.image.copy()

        thickness, color = 1, (0, 0, 255)
        self._displayPoly(image, self.upperLeftPoly, color, thickness)
        self._displayPoly(image, self.lowerRightPoly, color, thickness)
        cv.imshow('card', image)

    def rotatedScaled(self, angle: float, scale: float):
        assert scale <= 1.0
        height, width, _ = self.image.shape
        center = ((width-1)/2, (height-1)/2)
        M = cv.getRotationMatrix2D(center, angle, scale)
        newImage = cv.warpAffine(self.image, M, (width, height))

        M = np.concatenate([M, [[0, 0, 1]]], axis=0)
        origExtents = self._buildRectanglePoly(0, 0, width, height)
        extents = self._transformPoly(origExtents, M)
        minX = min(a[0] for a in extents)
        minY = min(a[1] for a in extents)
        maxX = max(a[0] for a in extents)
        maxY = max(a[1] for a in extents)

        croppedImage = newImage[minY:maxY+1, minX:maxX+1]
        card = Card(self.suit, self.value, croppedImage)
        card.upperLeftPoly = self._transformPoly(self.upperLeftPoly, M)
        card.upperLeftPoly = list((a[0]-minX, a[1]-minY)
                                  for a in card.upperLeftPoly)
        card.lowerRightPoly = self._transformPoly(self.lowerRightPoly, M)
        card.lowerRightPoly = list((a[0]-minX, a[1]-minY)
                                   for a in card.lowerRightPoly)

        return card


class Deck:

    def __init__(self):
        """ Loads the deck from the master PNG file """
        img = cv.imread('data/all-cards-cropped.png', cv.IMREAD_UNCHANGED)
        imgGray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

        rowBoundaries = Deck._findBoundaries(imgGray[:, 200])
        colBoundaries = Deck._findBoundaries(imgGray[20, :])
        suitLabels = ['C', 'H', 'S', 'D']
        valLabels = ['A', '2', '3', '4', '5', '6',
                     '7', '8', '9', 'T', 'J', 'Q', 'K']

        self.cardList = list()

        for (xMin, xMax), valLabel in zip(colBoundaries, valLabels):
            for (yMin, yMax), suitLabel in zip(rowBoundaries, suitLabels):
                image = img[yMin:yMax, xMin:xMax]
                self.cardList.append(Card(suitLabel, valLabel, image))

    @classmethod
    def _findBoundaries(cls, range):
        range = np.array(list(1 if a > 0 else 0 for a in range))
        rangeDiff = range[1:] - range[:-1]
        rangeEdges = np.array(
            list(a for a, b in enumerate(rangeDiff) if b != 0))
        return list(zip(rangeEdges[::2], rangeEdges[1::2]))

    def getRandom(self):
        return self.cardList[randint(0, len(self.cardList)-1)]


class BackgroundImage:

    def __init__(self, fileName: str):
        self._fileName = fileName
        self._image = None

    def getImage(self):
        if self._image is None:
            self._image = cv.imread(self._fileName)
            # self._image = cv.resize(
            #     self._image,
            #     (self._image.shape[0]*2, self._image.shape[1]*2),
            #     interpolation = cv.INTER_CUBIC)
        return self._image

    @classmethod
    def loadAll(cls):
        fileList = list(
            f for s in glob("../dtd/images/*") for f in glob(f"{s}/*.jpg"))
        print(f"Loaded {len(fileList)} images")
        return list(BackgroundImage(f) for f in fileList)


class BackgroundImageSet:

    def __init__(self):
        fileList = list(
            f for s in glob("../dtd/images/*") for f in glob(f"{s}/*.jpg"))
        print(f"Loaded {len(fileList)} images")
        self._imageList = list(BackgroundImage(f) for f in fileList)

    def getRandom(self):
        return self._imageList[randint(0, len(self._imageList)-1)]


class Scene:

    def __init__(self, backgroundImage: BackgroundImage, cardList: list):
        self.backgroundImage = backgroundImage
        self.cardList = cardList
        self.image = backgroundImage.getImage()

    def display(self):
        final = self.image.copy()
        print(final.shape)
        for card in self.cardList:
            card = card.rotatedScaled(30, 0.5)
            height, width, _ = card.image.shape
            alpha = card.image[:, :, 3] / 255.0
            for color in range(3):
                background = final[:height, :width, color] * (1-alpha)
                change = card.image[:, :, color] * alpha
                final[:height, :width, color] = \
                    background.astype(np.uint8) + change.astype(np.uint8)

        # final = cv.cvtColor(final, cv.COLOR_BGR2RGB)
        cv.imshow('scene', final)


deck = Deck()
card = deck.getRandom()
# card.display()
# card.rotatedScaled(30, 0.5).display()

backgroundImageSet = BackgroundImageSet()
scene = Scene(backgroundImageSet.getRandom(), [card])
scene.display()
cv.waitKey(0)
